## Notebooks to do tests on Constrained Brownian Motion

Code for trajectory generation is in constrainedGeneration.py

In [103]:
from constrainedSimulation import *
from helpersGeneration import *


%load_ext autoreload
%reload_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def animate_particle_trajectory_with_rectangle(trajectory, nframes, rect, points_per_frame=None, fps=5, save_path=None):
    """
    Creates an animation of a particle's trajectory inside a rectangular boundary, showing one subtrajectory per frame.
    Ensures equal axis scaling for correct proportions.
    
    Parameters:
    - trajectory: np.ndarray of shape (N, 2), where N is the total number of points.
                 Each row represents the (x, y) coordinates of the particle.
    - nframes: int, number of frames to divide the trajectory into.
    - rect: tuple (width, height), defining the size of the rectangle centered at (0,0).
    - points_per_frame: int, optional, number of points to show in each frame.
    - fps: int, frames per second for the animation.
    - save_path: str, optional, path to save the animation (supports .mp4 and .gif).
    
    Returns:
    - HTML animation that can be displayed in a Jupyter notebook.
    """
    fig, ax = plt.subplots(figsize=(5, 5))
    
    # Update the anchor point and rotation of the rectangle with the correct translation
    length, width, angle = rect  # Extract rectangle dimensions

    # Set the rectangle's center to (0, 0), and rotate it according to the angle
    rect_patch = plt.Rectangle((-width / 2 * np.cos(-angle) -length / 2 * np.sin(angle) , -length / 2 * np.cos(-angle) - width/ 2 * np.sin(-angle)), width, length, 
                            angle=-angle * 180 / np.pi, edgecolor='red', 
                            facecolor='none', linewidth=2)

    ax.add_patch(rect_patch)
    
    if points_per_frame is None:
        points_per_frame = len(trajectory) // nframes
    
    # Ensure equal scaling for both axes
    max_dim = max(width, length) / 2 + 1  # Add some padding
    ax.set_xlim([-max_dim, max_dim])
    ax.set_ylim([-max_dim, max_dim])
    ax.set_aspect('equal')  # Ensures square pixels
    ax.set_xticks([])
    ax.set_yticks([])
    
    line, = ax.plot([], [], lw=2, color='blue')
    frame_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12)
    x_data, y_data = [], []
    
    def init():
        line.set_data([], [])
        frame_text.set_text('')
        return line, frame_text
    
    def update(frame):
        start = frame * points_per_frame
        end = min(start + points_per_frame, len(trajectory))
        current_segment = trajectory[start:end]
        
        if frame == 0:
            x_data.clear()
            y_data.clear()
        
        x_data.extend(current_segment[:, 0])
        y_data.extend(current_segment[:, 1])
        
        line.set_data(x_data, y_data)
        frame_text.set_text(f'Frame {frame + 1}/{nframes}')
        
        return line, frame_text
    
    ani = animation.FuncAnimation(fig, update, frames=nframes,
                                 init_func=init, blit=True, interval=1000/fps)
    
    if save_path:
        if save_path.endswith('.mp4'):
            writer = animation.FFMpegWriter(fps=fps, metadata=dict(artist='Me'), bitrate=1800)
            ani.save(save_path, writer=writer)
        elif save_path.endswith('.gif'):
            writer = animation.PillowWriter(fps=fps)
            ani.save(save_path, writer=writer)
    
    plt.close()
    display(HTML(ani.to_jshtml()))

In [ ]:
# Tests on standard settings:
T = 200
D = 1
alpha = 1
deltaT = 1
# number of subposition per Frame for image Generation
nPosFrame = 10


# rectangle is given in (height, width, angle (rad))
rect = (50,4,0)
startPos = (0,0)

traj = traj_in_rectangle(T, D, alpha, deltaT, rect, startPos)
animate_particle_trajectory_with_rectangle(traj, T//nPosFrame, rect, nPosFrame)


In [ ]:
# Tests on startingPos:

startPos2 = (0,-15)

traj = traj_in_rectangle(T, D, alpha, deltaT, rect, startPos2)
animate_particle_trajectory_with_rectangle(traj, T//nPosFrame, rect, nPosFrame)
# Tests on startingPos:

startPos2 = (2,10)

traj = traj_in_rectangle(T, D, alpha, deltaT, rect, startPos2)
animate_particle_trajectory_with_rectangle(traj, T//nPosFrame, rect, nPosFrame)



In [ ]:
# Tests on angle and rectangle:

rect = (50,4,np.pi/2)
startPos = (0,0)

traj = traj_in_rectangle(T, D, alpha, deltaT, rect, startPos)
animate_particle_trajectory_with_rectangle(traj, T//nPosFrame, rect, nPosFrame)

# Tests on angle and rectannge:

rect = (50,4,-np.pi/6)
startPos = (0,0)

traj = traj_in_rectangle(T, D, alpha, deltaT, rect, startPos)
animate_particle_trajectory_with_rectangle(traj, T//nPosFrame, rect, nPosFrame)



In [ ]:
# Tests of image creation:
D_im = 0.5

rect = (50,4,np.pi)
startPos = (0,0)

traj = traj_in_rectangle(T, D_im, alpha, deltaT, rect, startPos)
animate_particle_trajectory_with_rectangle(traj, T//nPosFrame, rect, nPosFrame)


trajs = np.expand_dims(traj, axis=0)

videos = trajectories_to_video(trajs,nPosFrame,
                                   center=False,
                                   image_props={"upsampling_factor":10,
                                            "background_intensity": [300,10],
                                            "particle_intensity": [300,10],
                                            "resolution": 100e-9,
                                            "trajectory_unit" : -1,
                                            "output_size": 50,
                                            "poisson_noise" : -1
                                              })

print(videos.shape)
play_video(videos[0])


(1, 20, 50, 50)
vmin: 258.2889709472656 vmax: 460.75384521484375 mean: 300.43


In [106]:
# Tests on angle and rectannge:
D_im = 0.5

rect = (50,4,np.pi/6)
startPos = (0,0)

traj = traj_in_rectangle(T, D_im, alpha, deltaT, rect, startPos)
animate_particle_trajectory_with_rectangle(traj, T//nPosFrame, rect, nPosFrame)


trajs = np.expand_dims(traj, axis=0)

videos = trajectories_to_video(trajs,nPosFrame,
                                   center=False,
                                   image_props={"upsampling_factor":10,
                                            "background_intensity": [300,10],
                                            "particle_intensity": [300,10],
                                            "resolution": 100e-9,
                                            "trajectory_unit" : -1,
                                            "output_size": 50,
                                            "poisson_noise" : -1
                                              })

print(videos.shape)
play_video(videos[0])


(1, 20, 50, 50)
vmin: 258.5730285644531 vmax: 470.8002014160156 mean: 300.44
